# Preprocessing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Load the dataset
file_path = '/home/diego_nbotelho/code/diegonbotelho/f1-tire-prediction/raw_data/all_races_df.csv'
all_races_df = pd.read_csv(file_path)

In [ ]:
pd.set_option('display.max_columns', None)
all_races_df

In [ ]:
# Display basic info
print("Dataset shape:", all_races_df.shape)

In [ ]:
# Missing Data Analysis
print("Missing data per column:")
print(all_races_df.isnull().sum())

In [ ]:
# Exclusion of features that are not necessary
columns_to_remove = [
    'DriverNumber',
    'PitOutTime',
    'PitInTime',
    'Sector1SessionTime',
    'Sector2SessionTime',
    'Sector3SessionTime',
    'SpeedI1',
    'SpeedI2',
    'SpeedFL',
    'SpeedST',
    'IsPersonalBest',
    'FreshTyre',
    'Team',
    'LapStartTime',
    'LapStartDate',
    'Deleted',
    'DeletedReason',
    'FastF1Generated',
    'IsAccurate',
    'WindDirection',
    'WindSpeed'
]

new_df = all_races_df.drop(columns=columns_to_remove)
new_df.head(1)

In [ ]:
print(new_df[['Sector1Time', 'Sector2Time', 'Sector3Time', 'LapTime']].dtypes)

In [ ]:
# Convert lap times to seconds
new_df['LapTime'] = new_df['LapTime'].dt.total_seconds()
new_df['Sector1Time'] = new_df['Sector1Time'].dt.total_seconds()
new_df['Sector2Time'] = new_df['Sector2Time'].dt.total_seconds()
new_df['Sector3Time'] = new_df['Sector3Time'].dt.total_seconds()

In [ ]:

# Missing Data Analysis
print("Missing data per column:")
print(new_df.isnull().sum())

In [ ]:
# pd.set_option('display.max_rows', None)
new_df.head(200)

In [ ]:
# Encontrar as linhas onde 'Position' é NaN
linhas_com_posicao_nula = new_df[new_df['Position'].isnull()]

# Exibir essas linhas
print(linhas_com_posicao_nula)